# 4.5. Pomiar jakości modeli regresji

Stworzyliśmy model, który w pewien sposób przewiduje jakość wina na podstawie stężenia alkoholu. Jak widzieliśmy na wykresie, daleko jest nam od perfekcji. Gdybyśmy teraz otrzymali inny model i musieli zdecydować który z nich działa lepiej, to naoczne stwierdzenie tego faktu jest niemożliwe. Potrzebujemy metody, która zwróci nam ocenę obydwu modeli, abyśmy wybrali ten lepszy.

## Proces ewaluacji, a proces nauczania

Popełniliśmy podstawowy błąd podczas uczenia poprzedniego modelu i teraz go naprawimy. Wykorzystaliśmy cały zbiór do nauczenia regresji liniowej i teraz nie mamy na czym sprawdzić jakości jej predykcji dla nowych danych. Typowy proces zakłada podział zbioru na co najmniej 2, a czasem nawet 3 podzbiory:

1. Zbiór treningowy
2. Zbiór walidacyjny
3. Zbiór testowy

Wykorzystamy tylko zbiór treningowy i testowy, ponieważ regresja liniowa nie jest parametryzowalna.

In [3]:
import pandas as pd

In [4]:
wine_quality_df = pd.read_parquet("./data/wine-quality.parquet")
wine_quality_df.sample(n=5)

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality,color_red,color_white,bound sulfur dioxide
1532,7.5,0.23,0.49,7.7,0.049,61.0,209.0,0.99410,3.14,0.30,11.1,7,0,1,148.0
2383,6.7,0.26,0.29,5.8,0.025,26.0,74.0,0.99290,3.28,0.53,11.0,6,0,1,48.0
548,12.4,0.35,0.49,2.6,0.079,27.0,69.0,0.99940,3.12,0.75,10.4,6,1,0,42.0
204,7.0,0.43,0.36,1.6,0.089,14.0,37.0,0.99615,3.34,0.56,9.2,6,1,0,23.0
535,9.1,0.22,0.24,2.1,0.078,1.0,28.0,0.99900,3.41,0.87,10.3,6,1,0,27.0


In [5]:
from sklearn.model_selection import train_test_split

In [7]:
train_df, test_df = train_test_split(wine_quality_df, test_size=0.2)
train_df.shape, test_df.shape

((5197, 15), (1300, 15))

In [8]:
from sklearn.linear_model import LinearRegression

In [9]:
lr = LinearRegression()
lr.fit(train_df[["alcohol"]], train_df["quality"])

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [10]:
lr.coef_, lr.intercept_

(array([0.32735383]), 2.3844524492584953)

## Metody pomiaru jakości

Istnieje kilka metod pomiaru jakości stworzonego modelu regresji i opierają się one typowo na pomiarze błędu, jaki średnio popełnia dany model. Sam błąd można jednak liczyć na kilka sposobów i stąd też różnorodność dostępnych metod.

Od razu zauważmy, że błąd możemy policzyć osobno dla zbioru treningowego i testowego. 

### MAE (Mean Absolute Error)

Jeśli błąd predykcji modelu będziemy liczyć jako wartość absolutną różnicy pomiędzy oczekiwaną wartością, a tą przewidzianą przez model, to wartość średniego błędu będzie obliczana w następujący sposób:

$$ MAE = \frac{ \sum_{i=0}^{N}{ | \overline{y_{i}} - y_{i} | } }{ N } $$

In [11]:
from sklearn.metrics import mean_absolute_error

In [12]:
train_pred = lr.predict(train_df[["alcohol"]])
mean_absolute_error(train_df["quality"], train_pred)

0.6194424181995732

In [14]:
test_pred = lr.predict(test_df[["alcohol"]])
mean_absolute_error(test_df["quality"], test_pred)

0.6155902885507313

### MSE (Mean Squared Error)

Kolejną z metod pomiaru błędu dla modelu jest MSE, czyli błąd średniokwadratowy.

$$ MSE = \frac{ \sum_{i=0}^{N}{ ( \overline{y_{i}} - y_{i} )^{2} } }{ N } $$

In [15]:
from sklearn.metrics import mean_squared_error

In [16]:
mean_squared_error(train_df["quality"], train_pred)

0.6133844987775265

In [17]:
mean_squared_error(test_df["quality"], test_pred)

0.6061661189126857

MSE uwypukla istnienie tzw. outlierów, gdyż podnosi do kwadratu błędy większe niż 1. Istnieje także metryka RMSE (Root Mean Squared Error), która likwiduje zjawisko uwypuklania dużych błędów i liczy się ją w następujący sposób:

$$ RMSE = \sqrt{MSE} = \sqrt{ \frac{ \sum_{i=0}^{N}{ ( \overline{y_{i}} - y_{i} )^{2} } }{ N } } $$

In [21]:
import numpy as np

In [24]:
mean_squared_error(train_df["quality"], train_pred, 
                   squared=False)

0.783188673805697

In [25]:
mean_squared_error(test_df["quality"], test_pred, 
                   squared=False)

0.7785667080685416